# Rekognition によるテキスト検出

## 必要なパッケージのインポート

In [ ]:
import io
import boto3
from PIL import Image, ImageDraw

## クライアントの作成

In [ ]:
client = boto3.client('textract', region_name='us-west-2')

## ローカル画像からの検出

In [ ]:
img_pil = Image.open('imgs/running.jpg')
img_pil

In [ ]:
img_bytes_io = io.BytesIO()
img_pil.save(img_bytes_io, format="jpeg")
img_bytes = img_bytes_io.getvalue()

response = client.analyze_document(
    Document={
        'Bytes': img_bytes
    },
    FeatureTypes=[ 
        'TABLES',
        'FORMS'
    ]
)

response

In [ ]:
def show_texts(img, blocks):
    img_width, img_height = img.size
    draw = ImageDraw.Draw(img)

    for block in blocks:
        if 'Text' in block.keys():
            print(block['BlockType'], ':', 
                  block['Text'], ':', str(round(block['Confidence'], 1)) + ' %')
        elif 'Relationships' in block.keys():
            text = ''
            confidence = 0
            for relationship in block['Relationships']:
                if relationship['Type'] == 'CHILD':
                    for id in relationship['Ids']:
                        for rel_block in blocks:
                            if rel_block['Id'] == id and 'Text' in rel_block.keys():
                                text = rel_block['Text']
                                confidence = rel_block['Confidence']
                                break
                        if text != '':
                            break
                    if text != '':
                            break
            if text != '':
                if block['BlockType'] == 'CELL':
                    print(block['BlockType'], ':',
                          block['RowIndex'], '行 :',
                          block['ColumnIndex'], '列 :',
                          text, ':', str(round(confidence, 1)) + ' %')
                else:
                    print(block['BlockType'], ':', text, ':', str(round(confidence, 1)) + ' %')
            else:
                if block['BlockType'] == 'CELL':
                    print(block['BlockType'], ':',
                          block['RowIndex'], '行 :',
                          block['ColumnIndex'], '列')
                else:
                    print(block['BlockType'])
        else:
            if block['BlockType'] == 'CELL':
                print(block['BlockType'], ':',
                      block['RowIndex'], '行 :',
                      block['ColumnIndex'], '列')
            else:
                print(block['BlockType'])
        box = block['Geometry']['BoundingBox']
        draw.rectangle(
            (box['Left'] * img_width, box['Top'] * img_height, # 左上座標
            (box['Left'] + box['Width']) * img_width, (box['Top'] + box['Height']) * img_height), # 右下座標
            outline=(255, 0, 0) # 色
        )

    return img

In [ ]:
show_texts(img_pil.copy(), response['Blocks'])